## Phase 2: Find Processing Techniques

In [3]:
%reload_ext autoreload
%autoreload 2

In [13]:
# connects to utils and run a test for connectivity
from P2_Preporcessing_Utils import Test_py 
from P2_Preporcessing_Utils import test_py  
print(Test_py("My_Test_1").print_(), "\n")
print(test_py("My_Test_1"))

Class: Try if python utils connects to notebook: My_Test_1 

Function: Try if python utils connects to notebook: My_Test_1


In [15]:
from P2_Preporcessing_Utils import data_loading 

### Setup Pipeline Workflow

#### Data Preparation and Preprocessing

In [17]:
train, test = data_loading("train_full.csv", "test_full.csv")

### Iteration 1: Finding best preprocessing technique using custom CNN